# Generate Audio

This notebook is used to generate audio from TTA models for further analysis.

In [14]:
from diffusers import AudioLDMPipeline
import torch
import IPython.display as ipd
import scipy
import json
from experiments.utils.set_torch_device import set_torch_device
from pathlib import Path
import pandas as pd

## Template Augmentation

This block generates audio with the AudioLDM TTA model. Sounds are generated for each template, 5 per UCS class.

In [15]:
device, _ = set_torch_device()

#repo_id = "cvssp/audioldm-s-full-v2"
repo_id = "cvssp/audioldm-l-full"
pipe = AudioLDMPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

c:\Users\olive\miniconda3\envs\audiocraft\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
unet\diffusion_pytorch_model.safetensors not found

Fetching 15 files:  13%|█▎        | 2/15 [00:01<00:08,  1.57it/s]
















In [29]:
# Get templates from settings
with open ('experiments/configs/temp_aug.json', 'r') as f:
    settings = json.load (f)

# Get UCS classes, to be used as prompts
with open ("experiments/" + settings['class_converter'], 'r') as f:
    ucs_classes = json.load(f)

templates = settings['template_augmentations']
class_names = ucs_classes['class_to_int'].keys()
# Create folder structure
folders = [template.strip().replace(" ", "_") + "/" + class_name if template != "" else "class/" + class_name for template in templates for class_name in class_names]
augmented_prompts = [template + class_name for template in templates for class_name in class_names]

In [30]:
augmented_prompts

['air',
 'aircraft',
 'alarms',
 'ambience',
 'animals',
 'archived',
 'beeps',
 'bells',
 'birds',
 'boats',
 'bullets',
 'cartoon',
 'ceramics',
 'chains',
 'chemicals',
 'clocks',
 'cloth',
 'communications',
 'computers',
 'creatures',
 'crowds',
 'designed',
 'destruction',
 'dirt & sand',
 'doors',
 'drawers',
 'electricity',
 'equipment',
 'explosions',
 'farts',
 'fight',
 'fire',
 'fireworks',
 'foley',
 'food & drink',
 'footsteps',
 'games',
 'geothermal',
 'glass',
 'gore',
 'guns',
 'horns',
 'human',
 'ice',
 'lasers',
 'leather',
 'liquid & mud',
 'machines',
 'magic',
 'mechanical',
 'metal',
 'motors',
 'movement',
 'musical',
 'natural disaster',
 'objects',
 'paper',
 'plastic',
 'rain',
 'robots',
 'rocks',
 'rope',
 'rubber',
 'scifi',
 'snow',
 'sports',
 'swooshes',
 'tools',
 'toys',
 'trains',
 'user interface',
 'vegetation',
 'vehicles',
 'voices',
 'water',
 'weapons',
 'weather',
 'whistles',
 'wind',
 'windows',
 'wings',
 'wood',
 'this is a sound of air'

In [65]:
# Generate and save 5 sounds from each template+class
n_samples = 10
sample_length = 5.0 # Seconds
sample_rate = 16000
inference_steps = 100
save_path = "D:/clap/generated_audio/audioldm/"
sound_data = {
    "path" : [],
    "template" : [],
    "class" : []
}
for template in templates:
    for class_name in class_names:
        # Create folder
        if template != "":
            formatted_template = template.strip().replace(" ", "_")
            current_folder = formatted_template + "/" + class_name 
        else:
            formatted_template = "class"
            current_folder = "class/" + class_name 

        #Path(save_path + current_folder + "/").mkdir(parents=True, exist_ok=True)
        for i in range(n_samples):
            file_name = f"{formatted_template}_{class_name}_{i}.wav"
            sound_data["path"].append(save_path + file_name)
            sound_data["template"].append(template.strip())
            sound_data["class"].append(class_name)
            prompt = template + class_name
            audio = pipe(prompt, num_inference_steps=inference_steps, audio_length_in_s=sample_length).audios[0]
            scipy.io.wavfile.write(save_path + current_folder + "/" + file_name, rate=sample_rate, data=audio)

100%|██████████| 100/100 [00:04<00:00, 23.90it/s]


In [63]:
audio = pipe("cat meowing and purring gently", num_inference_steps=100, audio_length_in_s=sample_length).audios[0]

100%|██████████| 100/100 [00:04<00:00, 23.91it/s]


In [64]:
ipd.Audio(audio, rate=16000)